# **PRÁCTICA 4:  Machine Learning - Hackathon**
### Universitat de València, Escola Tecnica Superior d'Enginyeria
### Elena Marrero Castellano | 3ª curso del Grado Ciencia de Datos

Descargamos las librerías necesarias

In [ ]:
import scipy
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn import decomposition
from sklearn.svm import SVR, LinearSVR
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing, decomposition 
from sklearn.decomposition import PCA, KernelPCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV

Descargamos nuestros datos

In [ ]:
Xtrain = pd.read_csv("X_train.csv.gz") 
ytrain = pd.read_csv("y_train.csv.gz") 
Xtest = pd.read_csv("X_test.csv.gz")
# Convertimos de dataframe a arrays numpy
Xtrain = Xtrain.values[:, 1:]
ytrain = ytrain.values[:, 1:]
Xtest = Xtest.values[:, 1:]

**Primer paso**, realizar el preprocesado de los datos

In [ ]:
# Aplicamos la escala de MinMax a nuestros datos, Este estimador escala y traduce cada característica individualmente 
# de modo que esté en el rango dado en el conjunto de entrenamiento, por ejemplo, entre cero y uno.
scaler = MinMaxScaler()
scaler2 = MinMaxScaler()

# Ajuste solo en el set de entrenamiento.
scaler.fit(Xtrain)
scaler2.fit(ytrain)

# Aplicamos la transformación tanto al conjunto de train como al conjunto de test.
X_train_scaled = scaler.transform(Xtrain)
X_test_scaled = scaler.transform(Xtest)
y_train_scaled = scaler2.transform(ytrain)

In [ ]:
# # 90 características son muchas características, por lo que intentamos reducir la dimensionalidad implementando PCA.
# # Primero normalizamos sus datos usando scikit StandardScaler. Este es un paso necesario para pca:
# # Primero dividimos los datos (en nuestro caso ya está hecho)
# # Segundo, creamos la función para estandarizar y la entrenamos
# scaler = preprocessing.StandardScaler()
# scaler2 = preprocessing.StandardScaler()

# # Tercero, entrenamos con los datos de X_train
# scaler.fit(X_train_scaled) 
# scaler2.fit(y_train_scaled) 

# # Cuarto, estandarizamos aplicando la función a los datos que queremos estandarizar
# X_train_estandarizado = scaler.transform(X_train_scaled)  
# X_test_estandarizado = scaler.transform(X_test_scaled) 
# y_train_estandarizado = scaler2.transform(y_train_scaled) 

In [ ]:
X_train_scaled

array([[0.78320419, 0.48413238, 0.56832644, ..., 0.42161882, 0.4980812 ,
        0.31204558],
       [0.65074972, 0.54905976, 0.52135006, ..., 0.44243678, 0.51418811,
        0.31348747],
       [0.48489469, 0.47768308, 0.40661454, ..., 0.44116637, 0.50206017,
        0.3098551 ],
       ...,
       [0.76135618, 0.55887676, 0.52385109, ..., 0.41233513, 0.50486791,
        0.38347757],
       [0.82004854, 0.58435993, 0.5161119 , ..., 0.42633983, 0.49607448,
        0.31501065],
       [0.7320315 , 0.57311822, 0.50345402, ..., 0.42404866, 0.493367  ,
        0.32243068]])

**Segundo paso**, extracción de características. Si queremos hacer algo básico podríamos simplemnte mirar la PCA o podemos probar los diferentes números de dimensiones para el PCA. O lo que sería más interesante, investigat algunas librerías utilizadas del sklearn.

LinearRegression() 

In [ ]:
# Necesitamos un modelo para nuestros experimentos. Usaremos un modelo sencillo de regresión lineal
lr = LinearRegression() 

# Vamos a entrenar y testear nuestro modelo usando solo el conjunto de entrenamiento.
lr.fit(Xtrain, ytrain) 

# Por defecto `score` calcula el índice de correlación de Pearson al cuadrado
R2 = lr.score(Xtrain, ytrain)

def rmse(y, yp):
    return np.sqrt(np.sum((y-yp)**2) / y.shape[0])

def mae(y, yp):
    return np.sum(np.abs((y-yp)) / y.shape[0])

yp = lr.predict(Xtrain) # Predecimos utilizando el conjunto de entrenamiento
print('R2', R2, 'RMSE', rmse(ytrain, yp), 'MAE', mae(ytrain, yp))

# Pasamos a DaraFrame
y_hat = pd.DataFrame(yp)
# Guardamos .csv (lo que hay que subir)
y_hat.to_csv('y_hat.csv')

R2 0.2354512029158744 RMSE 9.626536091949268 MAE 6.84273834540552


PCA

In [ ]:
# Nuestros valores están normalizados, por lo que ahora aplicamos PCA
# Crea una instancia del modelo
pca = PCA(.90)
# PCA (.90) significa que scikit-learn elige el número mínimo de componentes principales de manera que se retiene el 
# 90% de la varianza.
pca.fit(X_train_scaled) 
# Imprime el número de componentes generados
print("En este caso, el 90% de la variación equivale a",pca.n_components_," componentes principales")

En este caso, el 90% de la variación equivale a 39  componentes principales


In [ ]:
# Sabiendo que tenemos 39 componentes principales:
model_pca = decomposition.PCA(n_components=39)
model_pca.fit(X_train_scaled) 
Xpca = model_pca.transform(X_train_scaled) 
ypca = y_train_scaled

In [ ]:
print(Xpca.shape)
print(ypca.shape)

(200000, 39)
(200000, 1)


In [ ]:
Xpca1 = Xpca[0:2000]
ypca1 = ypca[0:2000]

In [ ]:
print(Xpca1.shape)
print(ypca1.shape)

(2000, 39)
(2000, 1)


In [ ]:
parameters = {'C': np.logspace(-3,3,10), 'epsilon':np.logspace(-2,2,10)}
clf = GridSearchCV(estimator = LinearSVR(), param_grid = parameters, n_jobs = -1,
                   cv = 10, verbose = 2, scoring = "neg_mean_absolute_error")
clf.fit(Xpca1, ypca1)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 897 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    8.2s finished
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=10, error_score=nan,
             estimator=LinearSVR(C=1.0, dual=True, epsilon=0.0,
                                 fit_intercept=True, intercept_scaling=1.0,
                                 loss='epsilon_insensitive', max_iter=1000,
                                 random_state=None, tol=0.0001, verbose=0),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]),
                         'epsilon': array([1.00000000e-02, 2.78255940e-02, 7.74263683e-02, 2.15443469e-01,
       5.99484250e-01, 1.66810054e+00, 4.64158883e+00, 1.29154967e+01,
       3.59381366e+01, 1.00000000e+02])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_absolute_error', verbose=2)

In [ ]:
print(clf.best_params_)
print(clf.best_score_)

{'C': 0.46415888336127775, 'epsilon': 0.01}
-0.07491340963374975


In [ ]:
# Usamos KFold de scikit-learn
nf = 5  # number of folds
kf_r2, kf_rmse, kf_mae = [], [], []  # Listas para ir almacenando los resultados
kf = KFold(n_splits=nf, shuffle=True, random_state=1234)

for train, test in kf.split(Xpca[2000:20000]):  # pregunta: daría igual poner kf.split(X,y)?
    
    Xtrain1, ytrain1 = Xpca[train, :], ypca[train].ravel()

    # 1. Obtén los datos de test
    Xtest1, ytest1 = Xpca[test, :], ypca[test]

    # 2. Entrena el modelo (fit) con el conjunto de entrenamiento
    svr = LinearSVR(C =  0.46415888336127775, epsilon =  0.01)
    svr.fit(Xtrain1, ytrain1)   
    
    # 3. Predice sobre los datos de test
    y_hat = svr.predict(Xtest1)
    y_hat = y_hat.reshape(3600, 1)

    # Hacemos la inversa de los datos normalizados 
    y_hat_inverse = scaler2.inverse_transform(y_hat)
    ytest_inverse = scaler2.inverse_transform(ytest1)
    
    # 4. Calcula las métricas r2, rmse y mae
    MAE = mae(ytest_inverse, y_hat_inverse)
    
    # 5. Añádelas a las listas kf_r2, kf_rmse y kf_mae
    kf_mae.append(MAE)

In [ ]:
print(np.mean(kf_mae))

6.671405649510561


Predecimos sobre los datos finales. Para ello tenemos que aplicar la PCA a X_test_scales (porque el modelo está entrenado para 39 componentes, no para 90 características). Luego, predecir para X_test_scaled_pca. Y lo que obtendremos será y_hat, pero habrá que aplicarle la inversa a la normalización para tener años con sentido.

In [ ]:
print(X_test_scaled)
print(X_test_scaled.shape)

[[0.69012964 0.4179841  0.5303848  ... 0.40986096 0.50266906 0.32358687]
 [0.58510291 0.52860714 0.67156418 ... 0.46266765 0.43141059 0.34495695]
 [0.79299831 0.5435699  0.49853235 ... 0.41657482 0.51168806 0.31806143]
 ...
 [0.55466567 0.49606336 0.33750292 ... 0.52259067 0.5481554  0.33009194]
 [0.54375529 0.295145   0.53587868 ... 0.44992196 0.50930841 0.34569016]
 [0.65231302 0.507323   0.42179861 ... 0.47794846 0.5020464  0.30612884]]
(200000, 90)


Aplicamos PCA

In [ ]:
X_test_scaled_pca = pca.transform(X_test_scaled)
# Comprobamos qe tiene 39 componentes
X_test_scaled_pca.shape

(200000, 39)

Predecimos

In [ ]:
y_hat = svr.predict(X_test_scaled_pca)
print(y_hat.shape)

# Reshape
y_hat = y_hat.reshape(200000, 1)
print(y_hat.shape)

(200000,)
(200000, 1)


Observamos que no son años con sentido las predicciones.

In [ ]:
y_hat

array([[0.8809593 ],
       [0.70549517],
       [0.90086073],
       ...,
       [0.91514191],
       [0.89598913],
       [0.86228607]])

In [ ]:
y_hat = scaler2.inverse_transform(y_hat)
# Redondeamos
y_hat = np.round(y_hat[:])
print(y_hat)

[[2000.]
 [1984.]
 [2001.]
 ...
 [2003.]
 [2001.]
 [1998.]]


In [ ]:
#Pasamos a DaraFrame
y_hat = pd.DataFrame(y_hat)
#Guardamos .csv (lo que hay que subir)
y_hat.to_csv('y_hat.csv')